In [ ]:
## INFOH515 Pyspark code
## Author: Gianluca Bontempi
## Pyspark implementation of the least-squares in the INFOH515 slides "Map-reduce analytics" 


In [1]:
import numpy as np
import pwd
import getpass
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum

# create an instance of SparkSession
spark=SparkSession.builder.appName('s.com').getOrCreate()
sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 17:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 17:14:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/14 17:14:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:

np.random.seed(1225)   

n=20 # number of features
N=1000 # number of samples


X= np.random.normal(loc=0, scale=1, size=N * n).reshape(N, n)
X=np.concatenate((np.ones((N,1)),X), axis=1)
Y=2+X[:,1]-3*X[:,7]+np.random.normal(loc=0, scale=0.1, size=N )
Y=Y.reshape(N, 1)

YX = np.concatenate((Y,X), axis=1)
rddYX=sc.parallelize(YX)



In [10]:

## MapReduce computation of X^T X and X^T Y
mrXtX=rddYX.map(lambda x: np.dot(np.mat(x[1:]).T,np.mat(x[1:]))).reduce(lambda x,y:x+y) 
mrXtY=rddYX.map(lambda x: x[1:]*x[0]).reduce(lambda x,y:x+y)
        
beta=np.linalg.inv(mrXtX)@mrXtY


In [11]:

## In-memory computation of X^T X and X^T Y
XtX=np.dot(X.T,X)
XtY=X.T@Y    

## Comparison of the two computations
print(np.sum(np.abs(XtX-mrXtX)))
print(np.sum(np.abs(XtY.ravel()-mrXtY.ravel())))


1.0826006757724826e-11
2.4575896873102465e-12


In [12]:
np.argsort(-np.abs(beta))

matrix([[ 7,  0,  1, 19, 10, 12,  5,  4, 18, 11, 16, 20,  9,  2,  8,  3,
          6, 15, 14, 13, 17]])